<a href="https://colab.research.google.com/github/Dhairya511/Credit-Card-Fraud-Detection/blob/main/FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
truenews = pd.read_csv('/DATASET-FAKE.csv')
fakenews = pd.read_csv('/DATASET-TRUE.csv')

In [3]:
fakenews.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:

truenews.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:

fakenews.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [6]:
truenews.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [7]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [8]:
news = pd.concat([truenews, fakenews])
news["Article"] = news["title"] + news["text"]
news.sample(frac = 1)

,title,text,subject,date,True/Fake,Article
18127,THE PURGE: NYC Mayor De Blasio to Review ‘All ...,THE PURGE CONTINUES THE NYC MAYOR has just ope...,left-news,"Aug 16, 2017",True,THE PURGE: NYC Mayor De Blasio to Review ‘All ...
9891,Milestones in the Supreme Court immigration case,(Reuters) - The U.S. Supreme Court on Monday h...,politicsNews,"April 18, 2016",Fake,Milestones in the Supreme Court immigration ca...
18725,CAUGHT ON VIDEO: DELTA PILOT Smacks Fighting W...,A video of two women fighting in the jetway an...,left-news,"Apr 29, 2017",True,CAUGHT ON VIDEO: DELTA PILOT Smacks Fighting W...
15529,OBAMA’S REVEALING LETTER TO BROTHER TELLS WHY ...,"Well, it s not like we didn t know this but it...",politics,"Jun 28, 2015",True,OBAMA’S REVEALING LETTER TO BROTHER TELLS WHY ...
6816,Biden does not see Trump undoing much of envir...,OTTAWA (Reuters) - U.S. Vice President Joe Bid...,politicsNews,"December 9, 2016",Fake,Biden does not see Trump undoing much of envir...
...,...,...,...,...,...,...
21227,COMMUNIST FILMMAKER MICHAEL MOORE Recruits Lef...,"The despicable anti-free speech, communist clo...",left-news,"Dec 12, 2015",True,COMMUNIST FILMMAKER MICHAEL MOORE Recruits Lef...
2810,WATCH: Bill Maher’s Hilarious Opening Monolog...,Bill Maher is definitely not afraid of Donald ...,News,"January 28, 2017",True,WATCH: Bill Maher’s Hilarious Opening Monolog...
18496,"In rare official appearance, Oman's ruler meet...",DUBAI (Reuters) - The ruler of Oman received I...,worldnews,"October 2, 2017",Fake,"In rare official appearance, Oman's ruler meet..."
9715,HILLARY FINALLY Makes Statement On Sexual Pred...,"After several days of silence, Hillary Clinton...",politics,"Oct 10, 2017",True,HILLARY FINALLY Makes Statement On Sexual Pred...


In [14]:
from nltk.corpus import stopwords
import string

In [15]:
def process_text(s):

    nopunc = [char for char in s if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    clean_string = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_string

In [9]:

news.sample(5)

,title,text,subject,date,True/Fake,Article
2614,PA Senator Issues An EPIC Challenge To Trump ...,"Recently, Donald Trump shocked even his critic...",News,"February 9, 2017",True,PA Senator Issues An EPIC Challenge To Trump ...
4719,Trump Just Gave The Most BIZARRE Response For...,Audience members and viewers watching MSNBC s ...,News,"September 8, 2016",True,Trump Just Gave The Most BIZARRE Response For...
14623,ARRESTED: Cops Tell Shocking Story Of Domestic...,Are the Palin s above the law in Alaska? Will ...,politics,"Jan 20, 2016",True,ARRESTED: Cops Tell Shocking Story Of Domestic...
17913,Vaccination begins in Bangladesh camps to head...,"COX S BAZAR, Bangladesh (Reuters) - The World ...",worldnews,"October 10, 2017",Fake,Vaccination begins in Bangladesh camps to head...
9754,WATCH: DIVISIVE FORMER First Lady MICHELLE OBA...,"Michelle Obama is hands-down, the most divisiv...",politics,"Oct 4, 2017",True,WATCH: DIVISIVE FORMER First Lady MICHELLE OBA...


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_))

In [ ]:
news_bow = bow_transformer.transform(news['Clean Text'])

In [ ]:
print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

In [ ]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
fakenews_detect_model = MultinomialNB().fit(news_tfidf, news['True/Fake'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

In [ ]:
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))

In [ ]:
from sklearn.cross_validation import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
pipeline.fit(news_train,text_train)

In [ ]:

predictions = pipeline.predict(news_test)

In [ ]:

print(classification_report(predictions,text_test))